参数管理

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[ 0.1105],
        [-0.0159]], grad_fn=<AddmmBackward0>)

In [2]:
# 获取模型或模型部分的状态字典
print(net[0].state_dict())

OrderedDict([('weight', tensor([[ 0.1089, -0.1097, -0.0281, -0.4051],
        [ 0.0504,  0.1868, -0.2534,  0.2164],
        [ 0.1454,  0.4354,  0.3797, -0.3737],
        [-0.2444,  0.2058,  0.4850, -0.4469],
        [ 0.2029,  0.2484, -0.1585, -0.2269],
        [ 0.4823, -0.0741, -0.1780,  0.4760],
        [-0.4182,  0.0209, -0.2330,  0.0735],
        [ 0.2836,  0.4582,  0.0946,  0.3221]])), ('bias', tensor([-0.3135, -0.2079,  0.3889,  0.4550, -0.2938, -0.2370,  0.0185,  0.2152]))])


In [3]:
print(net[1].state_dict())

OrderedDict()


In [4]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2197, -0.0299, -0.0822,  0.2614,  0.1706, -0.2312,  0.0318,  0.3475]])), ('bias', tensor([-0.2960]))])


In [5]:
print(net.state_dict())

OrderedDict([('0.weight', tensor([[ 0.1089, -0.1097, -0.0281, -0.4051],
        [ 0.0504,  0.1868, -0.2534,  0.2164],
        [ 0.1454,  0.4354,  0.3797, -0.3737],
        [-0.2444,  0.2058,  0.4850, -0.4469],
        [ 0.2029,  0.2484, -0.1585, -0.2269],
        [ 0.4823, -0.0741, -0.1780,  0.4760],
        [-0.4182,  0.0209, -0.2330,  0.0735],
        [ 0.2836,  0.4582,  0.0946,  0.3221]])), ('0.bias', tensor([-0.3135, -0.2079,  0.3889,  0.4550, -0.2938, -0.2370,  0.0185,  0.2152])), ('2.weight', tensor([[ 0.2197, -0.0299, -0.0822,  0.2614,  0.1706, -0.2312,  0.0318,  0.3475]])), ('2.bias', tensor([-0.2960]))])


In [6]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2960], requires_grad=True)
tensor([-0.2960])


In [7]:
net[2].weight.grad == None

True

In [8]:
# named_paremeters生成一个迭代器
print([(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [9]:
net.state_dict()['2.bias'].detach()

tensor([-0.2960])

In [10]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        # 'block 0' -> 'block 1' -> 'block 2' -> 'block3'
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.1813],
        [-0.1814]], grad_fn=<AddmmBackward0>)

In [11]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [12]:
rgnet[0][1][0].bias.detach(), rgnet[0][1][0].weight.detach()

(tensor([ 0.4320, -0.3934,  0.3021, -0.0565, -0.3349, -0.4795, -0.4976, -0.3622]),
 tensor([[ 4.0120e-01, -2.5567e-01,  4.5978e-01, -4.5024e-01],
         [ 6.6828e-02, -1.1123e-01, -4.0444e-01,  2.7402e-01],
         [-7.1113e-02,  4.9153e-01,  4.5514e-01, -1.2342e-01],
         [ 1.9852e-01, -4.7397e-01,  9.0718e-05, -1.8572e-01],
         [ 1.7333e-01, -3.0725e-01,  1.3581e-01, -6.9968e-02],
         [-7.4567e-02, -4.3320e-01,  1.6600e-01, -2.9206e-01],
         [ 2.8228e-01,  2.1105e-01,  7.8039e-03, -1.0287e-01],
         [-4.5947e-01, -5.4411e-02, -3.7173e-01, -3.4874e-01]]))

In [13]:
def init_normal(m):
    if type(m) == nn.Linear:
        # 正态分布初始化
        nn.init.normal_(m.weight, mean=0, std=0.01)
        # make bias zero
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0153, -0.0072, -0.0017,  0.0066]), tensor(0.))

In [14]:
def init_constant(m):
    if type(m) == nn.Linear:
        # init a constant
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [15]:
def init_xavier(m):
    if type(m) == nn.Linear:
        #  Xavier 初始化
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5699,  0.2463, -0.0570, -0.5014])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [16]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        # 将张量初始化为指定范围内的均匀分布随机值
        nn.init.uniform_(m.weight, -10, 10)
        # 对于值为 False 的位置，对应位置上的权重值将被设置为 0
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 8.8516, -0.0000,  0.0000,  7.4828],
        [-5.5703, -0.0000,  0.0000, -9.8820]], grad_fn=<SliceBackward0>)

In [17]:
# we can change the value of parameter everytime
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000,  8.4828])

In [18]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
